In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD, IncrementalPCA
from scipy.sparse import csr_matrix, lil_matrix
import scipy
from itertools import chain
import statistics
from sklearn.pipeline import make_pipeline
from sklearn import svm, preprocessing
from sklearn.preprocessing import StandardScaler, normalize
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from numpy import expand_dims
from tensorflow.keras.utils import to_categorical
from tensorflow import concat
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import random
import tensorflow
import os
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras import layers, Input
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg19 import preprocess_input
from tensorflow.image import rgb_to_grayscale, grayscale_to_rgb
from tensorflow import tile
from tqdm import tqdm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
image_size = 224
input_shape = (image_size, image_size, 3)

In [3]:
# Data import

drawings_names = os.listdir('img/')
seasons = ['Autumn','Winter','Spring','Summer']
i = 0
drawings = []
labels = list()

for img in tqdm(drawings_names):
    if not img.startswith('.'):
        for season in seasons:
            if season in img:
                labels.append(str(season))
        img = load_img('img/'+img, target_size=(224,224,3))
        img = img_to_array(img)
        img = expand_dims(img,axis=0)
        img = preprocess_input(img)
        img = np.array(img)
        drawings.append(img)
        i = i + 1

100%|██████████| 1300/1300 [01:46<00:00, 12.17it/s]


In [5]:
# import VGG19 to extract the activations at conv layers
vgg = tensorflow.keras.applications.VGG19(weights='imagenet', include_top=True, input_shape = (224,224,3))
ixs = []

i=0
for layer in vgg.layers:
    if "conv" in layer.name:
        ixs.append(i)
    i=i+1

outputs = [vgg.layers[i].output for i in ixs]
feature_extraction_model = Model(inputs=vgg.input, outputs=outputs)

kernels_size = []
for i in range(len(feature_extraction_model.outputs)):
    kernels_size.append((feature_extraction_model.outputs[i].shape[1],feature_extraction_model.outputs[i].shape[2]))

# nombre de channels par couche
nb_channels = []
for i in range(len(feature_extraction_model.outputs)):
    nb_channels.append(feature_extraction_model.outputs[i].shape[3])

In [6]:
for i in range(len(ixs)):
    print('Indice ' + str(i) + ' : The length of a vector from layer ' + str(ixs[i]) + ' for a given image is : ' + str(kernels_size[i][0]**2 * nb_channels[i]) +
          ' (kernel size : ' + str(kernels_size[i][0]) + 'x' + str(kernels_size[i][0]) + ' and number of channels : ' + str(nb_channels[i]) + ')')

Indice 0 : La longueur d'un vecteur de la couche 1 pour une image donnée est de : 3211264 (taille kernel : 224x224 et nombre channels : 64)
Indice 1 : La longueur d'un vecteur de la couche 2 pour une image donnée est de : 3211264 (taille kernel : 224x224 et nombre channels : 64)
Indice 2 : La longueur d'un vecteur de la couche 4 pour une image donnée est de : 1605632 (taille kernel : 112x112 et nombre channels : 128)
Indice 3 : La longueur d'un vecteur de la couche 5 pour une image donnée est de : 1605632 (taille kernel : 112x112 et nombre channels : 128)
Indice 4 : La longueur d'un vecteur de la couche 7 pour une image donnée est de : 802816 (taille kernel : 56x56 et nombre channels : 256)
Indice 5 : La longueur d'un vecteur de la couche 8 pour une image donnée est de : 802816 (taille kernel : 56x56 et nombre channels : 256)
Indice 6 : La longueur d'un vecteur de la couche 9 pour une image donnée est de : 802816 (taille kernel : 56x56 et nombre channels : 256)
Indice 7 : La longueur d

## Run for every depth (except 0 and 1)

In [ ]:
depth = 15 # matches with the index above
outputs = vgg.layers[ixs[depth]].output
feature_extraction_model = Model(inputs=vgg.input, outputs=outputs)
nb_channel = feature_extraction_model.outputs[0].shape[3]
kernel_size = feature_extraction_model.outputs[0].shape[1]

In [ ]:
features_row = np.zeros((len(drawings),kernel_size**2 * nb_channel), dtype = 'float16')

for i in tqdm(range(len(drawings))): # for each image, extract features
    tmp_drawing = drawings[i]
    tmp_feature = feature_extraction_model.predict(tmp_drawing)
    flattened_features = []
    for j in range(nb_channel): # flattening of each feature that becomes a line of the matrix (for this layer)
        flattened_features.append(tmp_feature[0,:,:,j].flatten())
    flattened_features = list(chain.from_iterable(flattened_features))
    features_row[i,:] = flattened_features

### Number of Gb

In [ ]:
str(features_row.nbytes/(1e+9)) + ' Gb'

### PCA

In [ ]:
scaler = StandardScaler()
standardized_features_row = scaler.fit_transform(features_row)
pca = PCA(n_components = 0.8)
X_pca = pca.fit_transform(standardized_features_row)
X_pca.shape

np.savez_compressed('features/PCA_table/features_PCA_' + str(depth) + '.npz', X_pca, delimiters=',')
# data = np.load to load
# data['arr_0'] to have the dataset

#### Plot explained variance + cumulative explained variance

In [ ]:
exp_var_pca = pca.explained_variance_ratio_

cum_sum_eigenvalues = np.cumsum(exp_var_pca)

plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('features/plot_variance/cumulative_' + str(depth))
plt.show()

In [ ]:
plt.step(range(0,len(exp_var_pca)), exp_var_pca, where='mid',label='Individual explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('features/plot_variance/variance_' + str(depth))
plt.show()

### SVC on the PCA to have the scores

In [ ]:
clf = SVC()
scores = cross_val_score(clf, X_pca, labels, cv=10)

In [ ]:
scores

#### Save scores for this layer

In [ ]:
np.savetxt('features/scores/score_' + str(depth) + '.csv', scores, delimiter=",")

## Code for layers 0 and 1 to sample 50% of the activations

In [7]:
depth = 1 # only for layer 0 and 1
outputs = vgg.layers[ixs[depth]].output
feature_extraction_model = Model(inputs=vgg.input, outputs=outputs)
nb_channel = feature_extraction_model.outputs[0].shape[3]
kernel_size = feature_extraction_model.outputs[0].shape[1]


# for layer 0 and 1, sample 50% :
sampled_values = random.sample(range(kernel_size**2 * nb_channel),kernel_size**2 * nb_channel//2)

features_row = np.zeros((len(drawings), len(sampled_values)), dtype = 'float16')


#features_row = np.zeros((len(drawings),kernel_size**2 * nb_channel), dtype = 'float16')

for i in tqdm(range(len(drawings))):
    tmp_drawing = drawings[i]
    tmp_feature = feature_extraction_model.predict(tmp_drawing)
    flattened_features = []
    for j in range(nb_channel):
        flattened_features.append(tmp_feature[0,:,:,j].flatten())
    flattened_features = list(chain.from_iterable(flattened_features))
    features_row[i,:] = flattened_features

100%|██████████| 1299/1299 [18:28<00:00,  1.17it/s]


In [9]:
# all features for each images, for layer 0 and 1
np.savez_compressed('features/PCA_table/features_full_' + str(depth) + '.npz', features_row, delimiters=',')

### Is a sample of 50% of the points enough?

In [8]:
features_row = np.load('features/PCA_table/features_full_1.npz')
features_row = features_row['arr_0']

FileNotFoundError: [Errno 2] No such file or directory: 'features/PCA_table/features_full_1.npz'

In [ ]:
# Sample 50% of points 10 000 times, do a t-test to see if this mean = total mean

In [10]:
moyenne = features_row.mean()

In [11]:
moyenne

114.7

In [60]:
moyenne_sample = []
for i in tqdm(range(50)):
    moyenne_tmp = []
    sampled = random.sample(range(kernel_size**2 * nb_channel), kernel_size**2 * nb_channel//2)
    features_array = [ np.array(features_row[:,s]) for s in sampled]
    x = np.mean(features_array)
    moyenne_sample.append(x)
    #print(moyenne_sample)

100%|██████████| 50/50 [03:21<00:00,  4.03s/it]


In [67]:
scipy.stats.ttest_1samp(moyenne_sample,moyenne) # p = 1.0 for layer 0

Ttest_1sampResult(statistic=0.5018856132284956, pvalue=0.6179957940162921)

### PCA on the features

In [ ]:
scaler = StandardScaler()
standardized_features_row = scaler.fit_transform(features_row)
pca = PCA(n_components = 0.8)
X_pca = pca.fit_transform(standardized_features_row)
X_pca.shape

np.savez_compressed('features/PCA_table/features_PCA_' + str(depth) + 'b' +  '.npz', X_pca, delimiters=',')

In [ ]:
exp_var_pca = pca.explained_variance_ratio_

cum_sum_eigenvalues = np.cumsum(exp_var_pca)

plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('features/plot_variance/cumulative_' + str(depth) + 'b')
plt.show()

In [ ]:
plt.step(range(0,len(exp_var_pca)), exp_var_pca, where='mid',label='Individual explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('features/plot_variance/variance_' + str(depth) + 'b')
plt.show()

In [ ]:
clf = SVC()
scores = cross_val_score(clf, X_pca, labels, cv=10)

np.savetxt('features/scores/score_' + str(depth) + 'b' + '.csv', scores, delimiter=",")

## Feature activation of the 2 FC layers

In [ ]:
depth = 24 # FC layers : depth 23 and 24
outputs = vgg.layers[depth].output
feature_extraction_model = Model(inputs=vgg.input, outputs=outputs)
length_fc = feature_extraction_model.outputs[0].shape[1]

In [ ]:
features_row = np.zeros((len(drawings),length_fc), dtype = 'float32')

for i in tqdm(range(len(drawings))):
    tmp_drawing = drawings[i]
    tmp_feature = feature_extraction_model.predict(tmp_drawing)
    features_row[i,:] = tmp_feature

In [ ]:
scaler = StandardScaler()
standardized_features_row = scaler.fit_transform(features_row)
pca = PCA(n_components = 0.8)
X_pca = pca.fit_transform(standardized_features_row)
X_pca.shape

np.savez_compressed('features/PCA_table/features_PCA_' + vgg.layers[depth].name + '.npz', X_pca, delimiters=',')

In [ ]:
exp_var_pca = pca.explained_variance_ratio_

cum_sum_eigenvalues = np.cumsum(exp_var_pca)

plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('features/plot_variance/cumulative_' + vgg.layers[depth].name)
plt.show()

In [ ]:
plt.step(range(0,len(exp_var_pca)), exp_var_pca, where='mid',label='Individual explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('features/plot_variance/variance_' + vgg.layers[depth].name)
plt.show()

In [ ]:
clf = SVC()
scores = cross_val_score(clf, X_pca, labels, cv=10)

np.savetxt('features/scores/score_' + vgg.layers[depth].name + '.csv', scores, delimiter=",")